In [1]:
#importing the necessary libray

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option("display.min_rows", 100)
pd.set_option("display.max_columns", None)
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import classification_report

url = "Employee Atrition.csv"
data = pd.read_csv(url)
data.head()

,Age,Resigned,BusinessTravel,DailyRate($),Department,DistanceFromHome(km),EducationField,EmployeeNumber,EnvironmentSatisfaction,Gender,HourlyRate,JobInvolvement,JobLevel,JobRole,JobSatisfaction,MaritalStatus,MonthlyIncome($),MonthlyRate($),NumCompaniesWorked,Over18,OverTime,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,Life Sciences,1,2,Female,94,3,2,Sales Executive,4,Single,5993,19479,8,Y,Yes,11,3,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,Life Sciences,2,3,Male,61,2,2,Research Scientist,2,Married,5130,24907,1,Y,No,23,4,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,Other,4,4,Male,92,2,1,Laboratory Technician,3,Single,2090,2396,6,Y,Yes,15,3,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,Life Sciences,5,4,Female,56,3,1,Research Scientist,3,Married,2909,23159,1,Y,Yes,11,3,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,Medical,7,1,Male,40,3,1,Laboratory Technician,2,Married,3468,16632,9,Y,No,12,3,4,80,1,6,3,3,2,2,2,2


In [2]:
# Split the original data into training and testing sets, keeping the categorical variables intact
from sklearn.model_selection import train_test_split

X_original = data.drop('Resigned', axis=1)  # Original features
y_original = data['Resigned']  # Already encoded target variable

In [3]:
#importing the necessary libraries
from collections import Counter
import imblearn
from imblearn.over_sampling import SMOTE

X_original_initial = X_original

X_original = pd.get_dummies(X_original, drop_first=True)

smote = SMOTE()

X_original_smote, y_original_smote = smote.fit_resample(X_original, y_original)
print("Original target shape: {}".format(Counter(y_original)))
print("Resampled target shape: {}".format(Counter(y_original_smote)))

Original target shape: Counter({'No': 1233, 'Yes': 237})
Resampled target shape: Counter({'Yes': 1233, 'No': 1233})


In [4]:
X_train_orig, X_test_orig, y_train_orig, y_test_orig = train_test_split(X_original_smote, y_original_smote, test_size=0.2, random_state=0)


X_train_encoded = pd.get_dummies(X_train_orig, drop_first=True)
X_test_encoded = pd.get_dummies(X_test_orig, drop_first=True)
y_test_orig_encoded = pd.get_dummies(y_test_orig, drop_first=True)
y_train_orig_encoded = pd.get_dummies(y_train_orig, drop_first=True)

# Ensure both train and test sets have the same columns after encoding
missing_cols = set(X_train_encoded.columns) - set(X_test_encoded.columns)
for c in missing_cols:
    X_test_encoded[c] = 0  # Add missing columns as zeros
X_test_encoded = X_test_encoded[X_train_encoded.columns]  # Reorder columns to match training set


#calling on the modelling function
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(random_state=0, n_estimators=200,criterion='entropy', n_jobs=-1, max_depth = 70, min_samples_leaf=12)


# Initialize and train the RandomForestClassifier with the encoded data
model.fit(X_train_encoded, y_train_orig_encoded)

# Predictions and full probabilities on the encoded test data
y_pred_orig = model.predict(X_test_encoded)
y_probs_orig = model.predict_proba(X_test_encoded)  # Getting probabilities for both classes

# Create a DataFrame combining original test data, actual y, predicted y, and probabilities for both classes
results_df_full = X_test_orig.copy()
results_df_full['Actual'] = y_test_orig
results_df_full['Predicted'] = y_pred_orig
results_df_full['Probability_Not_Resigned'] = y_probs_orig[:, 0]  # Probability of the negative class (not resigning)
results_df_full['Probability_Resigned'] = y_probs_orig[:, 1]  # Probability of the positive class (resigning)

results_df_full.head(1000)


Training_Accuracy = model.score(X_train_encoded, y_train_orig_encoded) * 100
Test_Accuracy = model.score(X_test_encoded, y_test_orig_encoded)* 100


In [5]:
results_df_full.head(10)

,Age,DailyRate($),DistanceFromHome(km),EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome($),MonthlyRate($),NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Male,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes,Actual,Predicted,Probability_Not_Resigned,Probability_Resigned
575,54,376,19,799,4,95,3,2,1,5485,22670,9,11,3,2,80,2,9,4,3,5,3,1,4,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,No,0,0.834379,0.165621
1096,40,898,6,1550,3,38,3,4,4,16437,17381,1,21,4,4,80,0,21,2,3,21,7,7,7,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,No,0,0.756560,0.243440
1791,19,1329,10,679,4,32,3,1,3,1035,26991,1,11,3,4,80,0,0,4,2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,Yes,1,0.267202,0.732798
715,33,827,1,998,3,84,4,2,2,5488,20161,1,13,3,1,80,1,6,2,3,6,5,1,2,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,No,0,0.782884,0.217116
1023,56,1255,1,1441,1,90,3,1,1,2066,10494,2,22,4,4,80,1,5,3,4,3,2,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,No,0,0.707170,0.292830
1512,30,825,5,1683,2,72,2,1,3,1200,16076,1,12,3,2,80,0,1,3,2,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,Yes,1,0.070228,0.929772
2210,34,1202,4,1095,3,85,3,3,1,9585,10343,0,21,3,1,80,0,9,2,3,7,6,3,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,Yes,1,0.217281,0.782719
1261,38,833,18,1766,2,60,1,2,4,5811,24539,3,16,3,3,80,1,15,2,3,1,0,1,0,0,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,1,No,0,0.689269,0.310731
1860,25,383,8,1433,1,68,2,1,1,4395,15180,2,11,3,1,80,0,5,2,2,2,1,1,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,Yes,1,0.125110,0.874890
2366,30,628,3,530,1,58,2,1,1,2574,21271,4,17,3,1,80,0,11,2,3,8,7,1,7,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,Yes,1,0.245801,0.754199


In [6]:
# Use replace to map values
results_df_full['Predicted'] = results_df_full['Predicted'].replace({0: 'No', 1: 'Yes'})

# Display the updated DataFrame
results_df_full.head()


,Age,DailyRate($),DistanceFromHome(km),EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome($),MonthlyRate($),NumCompaniesWorked,PercentSalaryHike,PerformanceRating,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Research & Development,Department_Sales,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,Gender_Male,JobRole_Human Resources,JobRole_Laboratory Technician,JobRole_Manager,JobRole_Manufacturing Director,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Married,MaritalStatus_Single,OverTime_Yes,Actual,Predicted,Probability_Not_Resigned,Probability_Resigned
575,54,376,19,799,4,95,3,2,1,5485,22670,9,11,3,2,80,2,9,4,3,5,3,1,4,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,1,No,No,0.834379,0.165621
1096,40,898,6,1550,3,38,3,4,4,16437,17381,1,21,4,4,80,0,21,2,3,21,7,7,7,0,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,0,1,1,No,No,0.756560,0.243440
1791,19,1329,10,679,4,32,3,1,3,1035,26991,1,11,3,4,80,0,0,4,2,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,Yes,Yes,0.267202,0.732798
715,33,827,1,998,3,84,4,2,2,5488,20161,1,13,3,1,80,1,6,2,3,6,5,1,2,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,No,No,0.782884,0.217116
1023,56,1255,1,1441,1,90,3,1,1,2066,10494,2,22,4,4,80,1,5,3,4,3,2,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,No,No,0.707170,0.292830


In [7]:
  #Accuracy metrics check
precision = precision_score(y_test_orig_encoded, y_pred_orig) * 100
recall = recall_score(y_test_orig_encoded, y_pred_orig) * 100
f_score = f1_score(y_test_orig_encoded, y_pred_orig) * 100

    
data = {'Model': ["model"],
        "Train_acc": [Training_Accuracy],
        "Test_acc": [Test_Accuracy],
        "Precision": [precision],
        "Recall": [recall],
        "F-score": [f_score]}
model_metrics = pd.DataFrame(data = data)

In [8]:
model_metrics

,Model,Train_acc,Test_acc,Precision,Recall,F-score
0,model,92.799189,87.449393,91.020408,84.790875,87.795276


In [9]:
#results_df_full.to_csv("Prediction_Test_Data.csv")